# Data Loading and Preprocessing

In [ ]:
import numpy as np
import pandas as pd
import re
import warnings
warnings.filterwarnings('ignore')

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Paths to the zip files in Drive
zip_dir = '/content/drive/MyDrive/jigsaw-toxic-comment-classification-challenge/'

# Unzip training and testing zips
import zipfile
with zipfile.ZipFile(zip_dir + 'train.csv.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/')
with zipfile.ZipFile(zip_dir + 'test.csv.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/')

# Load CSV files
train_df = pd.read_csv('/content/train.csv')
test_df = pd.read_csv('/content/test.csv')

print("Training dataset shape:", train_df.shape)
print("Test dataset shape:", test_df.shape)

# Text cleaning function
def clean_text(text):
    text = text.lower()
    text = re.sub(r'http\S+|www\S+|https\S+', '', text)  # Remove URLs
    text = re.sub(r'\S+@\S+', '', text)                  # Remove emails
    text = re.sub(r'[^a-z\s]', '', text)                 # Remove special characters and numbers
    text = re.sub(r'\s+', ' ', text).strip()             # Remove extra spaces
    return text

# Clean the comments
train_df['clean_comment'] = train_df['comment_text'].apply(clean_text)
test_df['clean_comment'] = test_df['comment_text'].apply(clean_text)

print(train_df[['comment_text', 'clean_comment']].head())


Mounted at /content/drive
Training dataset shape: (159571, 8)
Test dataset shape: (153164, 2)
                                        comment_text  \
0  Explanation\nWhy the edits made under my usern...   
1  D'aww! He matches this background colour I'm s...   
2  Hey man, I'm really not trying to edit war. It...   
3  "\nMore\nI can't make any real suggestions on ...   
4  You, sir, are my hero. Any chance you remember...   

                                       clean_comment  
0  explanation why the edits made under my userna...  
1  daww he matches this background colour im seem...  
2  hey man im really not trying to edit war its j...  
3  more i cant make any real suggestions on impro...  
4  you sir are my hero any chance you remember wh...  


# Feature Extraction with TF‑IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Create vectorizer
vectorizer = TfidfVectorizer(max_features=10000, stop_words='english')
X_train = vectorizer.fit_transform(train_df['clean_comment'])
X_test = vectorizer.transform(test_df['clean_comment'])

print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)

# Define toxicity labels
label_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
y_train = train_df[label_cols].values


Shape of X_train: (159571, 10000)
Shape of X_test: (153164, 10000)


# Train/Validation Split and SVM Training

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.calibration import CalibratedClassifierCV

# Split the data - 80/20 Split
X_train_split, X_val, y_train_split, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Set up a base LinearSVC and wrap it with calibration for probability estimates
base_svm = LinearSVC(random_state=42)

# Use CalibratedClassifierCV to get probabilities
calibrated_svm = OneVsRestClassifier(CalibratedClassifierCV(base_svm, cv=3))

# Train the classifier on the training split
calibrated_svm.fit(X_train_split, y_train_split)

# Predict on the validation set
y_val_prob = calibrated_svm.predict_proba(X_val)
y_val_pred = calibrated_svm.predict(X_val)

print("Validation predictions shape:", y_val_pred.shape)


Validation predictions shape: (31915, 6)


# Evaluation

In [ ]:
import numpy as np
from sklearn.metrics import (
    precision_recall_fscore_support,
    accuracy_score,
    roc_auc_score,
    average_precision_score,
    confusion_matrix,
    f1_score,
)

# Define labels and thresholds
label_cols = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
optimized_thresholds = {
    "toxic": 0.5082,
    "severe_toxic": 0.5408,
    "obscene": 0.6551,
    "threat": 0.1,
    "insult": 0.5082,
    "identity_hate": 0.1
}
optimized_thresh_array = np.array([optimized_thresholds[lbl] for lbl in label_cols])
print("Optimized thresholds array:", optimized_thresh_array)

# Use the predictions from the calibrated classifier
all_probs = y_val_prob  # Predicted probabilities for validation set
all_labels = y_val      # Ground truth labels

# Binarize predictions using thresholds
bin_preds = (all_probs >= optimized_thresh_array).astype(int)

# Compute per‑label Precision, Recall, and F1 scores
prec, rec, f1, _ = precision_recall_fscore_support(all_labels, bin_preds, average=None, zero_division=0)
macro_f1 = np.mean(f1)

# Compute micro‑averaged metrics
micro_p, micro_r, micro_f1, _ = precision_recall_fscore_support(
    all_labels.ravel(), bin_preds.ravel(), average="micro", zero_division=0
)

# Compute subset accuracy
subset_acc = accuracy_score(all_labels, bin_preds)

# Compute ROC‑AUC and PR‑AUC
roc_auc_macro = roc_auc_score(all_labels, all_probs, average="macro")
pr_auc_macro = average_precision_score(all_labels, all_probs, average="macro")

# Compute and print per‑label Confusion Matrices
conf_matrices = {}
for i, lbl in enumerate(label_cols):
    cm = confusion_matrix(all_labels[:, i], bin_preds[:, i])
    conf_matrices[lbl] = cm
    print(f"Confusion matrix for {lbl}:")
    print(cm)
    print()

# Print Report
print("\n===== Evaluation Metrics =====")
for i, lbl in enumerate(label_cols):
    print(f"{lbl:15s}  Precision: {prec[i]:.3f}  Recall: {rec[i]:.3f}  F1: {f1[i]:.3f}")
print("-------------------------------------------------")
print(f"Macro‑F1           : {macro_f1:.4f}")
print(f"Micro‑F1           : {micro_f1:.4f}")
print(f"Subset accuracy    : {subset_acc:.4f}")
print(f"ROC‑AUC (macro)    : {roc_auc_macro:.4f}")
print(f"PR‑AUC  (macro)    : {pr_auc_macro:.4f}")
print("=================================================\n")


Optimized thresholds array: [0.5082 0.5408 0.6551 0.1    0.5082 0.1   ]
Confusion matrix for toxic:
[[28600   259]
 [ 1075  1981]]

Confusion matrix for severe_toxic:
[[31562    32]
 [  264    57]]

Confusion matrix for obscene:
[[30110    90]
 [  673  1042]]

Confusion matrix for threat:
[[31770    71]
 [   36    38]]

Confusion matrix for insult:
[[30105   196]
 [  758   856]]

Confusion matrix for identity_hate:
[[31394   227]
 [  141   153]]


===== Evaluation Metrics =====
toxic            Precision: 0.884  Recall: 0.648  F1: 0.748
severe_toxic     Precision: 0.640  Recall: 0.178  F1: 0.278
obscene          Precision: 0.920  Recall: 0.608  F1: 0.732
threat           Precision: 0.349  Recall: 0.514  F1: 0.415
insult           Precision: 0.814  Recall: 0.530  F1: 0.642
identity_hate    Precision: 0.403  Recall: 0.520  F1: 0.454
-------------------------------------------------
Macro‑F1           : 0.5449
Micro‑F1           : 0.9800
Subset accuracy    : 0.9157
ROC‑AUC (macro)    : 0.